In [3]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import keras

In [5]:
# Declare dataset path
dataset_folder = "C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48"
sub_folders = os.listdir(dataset_folder)

# Read folder names as labels and images underneath
i = 0
last = []
images = []
labels = []
temp = sub_folders

# Read folders in the main dataset folder, one at a time
for sub_folder in sub_folders:
    sub_folder_index = temp.index(sub_folder)
    label = sub_folder_index
    # # Define labels basis use case. We are using positive:0, negative:1, neutral:2
    # # for our use case of predicting emotions of visitors entering a retail store
    # if label in [4, 6]:  # label in ['happy', 'surprise']
    #     new_label = 0  # changed to label = positive emotion
    # elif label in [0, 5]:  # label in ['anger','sadness']
    #     new_label = 1  # changed to label = negative emotion
    # else:  # label in ['contempt', 'disgust', 'fear']
    #     new_label = 2  # changed to label = neutral emotion

    path = dataset_folder + "/" + sub_folder
    sub_folder_images = os.listdir(path)

    # Read images in the sub folder, one at a time
    for image in sub_folder_images:
        image_path = path + "/" + image
        print(image_path + "\t" + str(label))
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (48, 48))
        images.append(image)
        labels.append(label)
        i += 1
    last.append(i)

# Declare x and y
images_x = np.array(images)
labels_y = np.array(labels)
# We divide image pixels by 255 to reduce computation power
images_x = images_x / 255

# encoding the labels
num_of_classes = 7
labels_y_encoded = keras.utils.to_categorical(labels_y, num_classes=num_of_classes)

# Split into 75:25 train and test
X_train, X_test, Y_train, Y_test = train_test_split(images_x, labels_y_encoded, test_size=0.25, random_state=10)
print(X_train, X_test, Y_train, Y_test)

C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S010_004_00000017.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S010_004_00000018.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S010_004_00000019.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S011_004_00000019.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S011_004_00000020.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S011_004_00000021.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S014_003_00000028.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S014_003_00000029.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S014_003_00000030.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S022_005_00000030.png	0
C:/Users/taylo/OneDrive/Documents/Dissertation Work/CK+48/anger/S022_005_00000031.png	0
C:/Users/taylo/OneDrive/Document

In [ ]:
from keras.layers import Input

In [ ]:
# CNN Architecture
input = Input(shape=(48, 48, 1))
conv1 = Conv2D(32, (3, 3), padding="same", strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation("relu")(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), padding="same", strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation("relu")(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), padding="same", strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation("relu")(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(256, (3, 3), padding="same", strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation("relu")(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
flatten = Flatten()(pool4)
dense_1 = Dense(128, activation="relu")(flatten)
drop_1 = Dropout(0.2)(dense_1)
output = Dense(7, activation="sigmoid")(drop_1)

# Model compile
model = Model(inputs=input, outputs=output)
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=["accuracy"])
model.summary()

# Configure Model Checkpoint
fle_s = "./output/emotion_model.keras"
checkpointer = ModelCheckpoint(
    fle_s,
    monitor="loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)
callback_list = [checkpointer]